In [65]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Question 1

In [82]:
# nc -l -p 9999 # to create a server on mac

#### then connect jupyter with spark (in terminal) 

cd spark-2.3.0-bin-hadoop2.7

export PYSPARK_DRIVER_PYTHON=jupyter

export PYSPARK_DRIVER_PYTHON_OPTS='notebook'

SPARK_LOCAL_IP=127.0.0.1 ./bin/pyspark

In [181]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql.functions import *

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
import pyspark.sql as sql


from pyspark.sql import SQLContext

spark = SparkSession \
    .builder \
    .appName("APP") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

sqlContext = SQLContext(spark)

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, ",")
   ).alias("word")
)

In [182]:
type(words)

pyspark.sql.dataframe.DataFrame

In [183]:
query1 = words \
    .writeStream \
    .queryName("Query1") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [ ]:
query1.stop()

In [185]:
Q1_3 = spark.sql("select * from Query1 order by word asc")
Q1_3.show()

+----+
|word|
+----+
| -20|
| 100|
|  40|
+----+



In [186]:
Q1_4 = spark.sql("select min(word) as minimum, max(word) as maximum, avg(word) as average from Query1")
Q1_4.show()

+-------+-------+-------+
|minimum|maximum|average|
+-------+-------+-------+
|    -20|     40|   40.0|
+-------+-------+-------+



# Question 2

In [187]:
# 1. type python server.py in terminal
# 2. type nc localhost 9999 in another terminal # client connect on mac

In [84]:
# code check server.py

# Question 3

In [188]:
from pyspark.sql.types import *

In [198]:
spark = SparkSession \
    .builder \
    .appName("Titanic") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

sqlContext = SQLContext(spark)

titanic = lines.withColumn("PassengerId", split(col("value"), ",").getItem(0).cast(IntegerType())).withColumn("Survived", split(col("value"), ",").getItem(1).cast(IntegerType())).withColumn("Pclass", split(col("value"), ",").getItem(2)).withColumn("Name", split(col("value"), ",").getItem(3)).withColumn("Sex", split(col("value"), ",").getItem(4)).withColumn("Age", split(col("value"), ",").getItem(5).cast(DoubleType())).withColumn("Fare", split(col("value"), ",").getItem(6).cast(DoubleType())).drop("value")
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)



In [199]:
type(titanic)

pyspark.sql.dataframe.DataFrame

In [200]:
query2 = titanic \
    .writeStream \
    .queryName("Query2") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [204]:
# Please wait for 10 seconds to run everything.
query2.stop()

In [201]:
Q3 = spark.sql("select * from Query2")
Q3.show(10)
#Q3.describe()

+-----------+--------+------+--------------------+------+----+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|   Fare|
+-----------+--------+------+--------------------+------+----+-------+
|          1|       0|     3| Mr. Owen Harris ...|  male|22.0|   7.25|
|          5|       0|     3| Mr. William Henr...|  male|35.0|   8.05|
|          9|       1|     3| Mrs. Oscar W (El...|female|27.0|11.1333|
|         13|       0|     3| Mr. William Henr...|  male|20.0|   8.05|
|         17|       0|     3| Master. Eugene Rice|  male| 2.0| 29.125|
|         21|       0|     2| Mr. Joseph J Fynney|  male|35.0|   26.0|
|         25|       0|     3| Miss. Torborg Da...|female| 8.0| 21.075|
|         29|       1|     3|" Miss. Ellen ""N...|female|null| 7.8792|
|          2|       1|     1| Mrs. John Bradle...|female|38.0|71.2833|
|          6|       0|     3|     Mr. James Moran|  male|null| 8.4583|
+-----------+--------+------+--------------------+------+----+-------+
only s

### 3. (1 mark) Write a SQL query to show the percentage of passengers who survived.

In [202]:
spark.sql("select sum(Survived)/count(Survived) *100 as Percentage from Query2").show()

+-----------------+
|       Percentage|
+-----------------+
|38.92694063926941|
+-----------------+



In [203]:
spark.sql("select sum(Survived) from Query2").show()
spark.sql("select count(Survived) from Query2").show()
# therefore there is a bit difference of the final answer and the given answer.

+-------------+
|sum(Survived)|
+-------------+
|          341|
+-------------+

+---------------+
|count(Survived)|
+---------------+
|            876|
+---------------+



### 4. Write a SQL query to show the number of passengers surviving under each gender category. When getting continuous data, did you see any pattern in the data

In [205]:
spark.sql("select sex, sum(Survived) as Survived from Query2 group by sex").show()

+------+--------+
|   sex|Survived|
+------+--------+
|female|     233|
|  male|     108|
+------+--------+



### Obviously there are much more female survived than male. So, you may have more chance to survive as a female in this case (like a disaster).

### 5. (1 mark) Write a SQL query to show the percentage of passengers who survived under each class category. Show the results in ascending order by `Pclass`.

In [207]:
Q3_5=spark.sql("select Pclass, count(Pclass)/(select count(*) from Query2) as Percentage from Query2 group by Pclass order by Pclass asc").show()

+------+-------------------+
|Pclass|         Percentage|
+------+-------------------+
|     1| 0.2408675799086758|
|     2|0.20319634703196346|
|     3| 0.5559360730593608|
+------+-------------------+



### Question 4

In [208]:
# please run serverQ4.py

In [209]:
spark = SparkSession \
    .builder \
    .appName("OMG") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

sqlContext = SQLContext(spark)

# Split the lines into words
words_TT = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("TTword")
)

In [210]:
type(words_TT)

pyspark.sql.dataframe.DataFrame

In [211]:
# select words contain #
words_TT = words_TT.where(words_TT.TTword.like("#%"))

In [212]:
query3 = words_TT \
    .writeStream \
    .queryName("Query3") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [214]:
# It takes a while to run everything. maybe 10 minute ish. Try drink a coffee and come back, it may still running.

In [215]:
query3.stop()

In [216]:
spark.sql("select TTword, count(*) as Count from Query3 group by TTword order by Count desc").show(10)

+------------+-----+
|      TTword|Count|
+------------+-----+
|     #Hiring|37964|
|       #Jobs|24776|
|  #CareerArc|21845|
|        #Job|21368|
|        #job|17763|
|     #Retail| 7867|
|#Hospitality| 7664|
|       #job?| 7569|
|    #hiring!| 6860|
|       #Job:| 5953|
+------------+-----+
only showing top 10 rows



# Thank you Matt!